In [2]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import linear_kernel
# from sklearn.feature_extraction.text import TfidVectorizer

In [3]:
df_movies=pd.read_csv("tmdb_5000_movies.csv")
df_credits=pd.read_csv("tmdb_5000_credits.csv")

In [23]:
df_movies.tail(2)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
4801,0,[],http://shanghaicalling.com/,126186,[],en,Shanghai Calling,When ambitious New York attorney Sam is sent t...,0.857008,[],"[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-05-03,0,98.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,A New Yorker in Shanghai,Shanghai Calling,5.7,7
4802,0,"[{""id"": 99, ""name"": ""Documentary""}]",NaN,25975,"[{""id"": 1523, ""name"": ""obsession""}, {""id"": 224...",en,My Date with Drew,Ever since the second grade when he first saw ...,1.929883,"[{""name"": ""rusty bear entertainment"", ""id"": 87...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2005-08-05,0,90.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,NaN,My Date with Drew,6.3,16


In [5]:
df_movies['overview']

0       In the 22nd century, a paraplegic Marine is di...
1       Captain Barbossa, long believed to be dead, ha...
2       A cryptic message from Bond’s past sends him o...
3       Following the death of District Attorney Harve...
4       John Carter is a war-weary, former military ca...
                              ...                        
4798    El Mariachi just wants to play his guitar and ...
4799    A newlywed couple's honeymoon is upended by th...
4800    "Signed, Sealed, Delivered" introduces a dedic...
4801    When ambitious New York attorney Sam is sent t...
4802    Ever since the second grade when he first saw ...
Name: overview, Length: 4803, dtype: object

In [6]:
tfidf=CountVectorizer(stop_words="english")
df_movies['overview']=df_movies['overview'].fillna('')
tfidf_matrix=tfidf.fit_transform(df_movies['overview'])

In [7]:
cosine_sim=linear_kernel(tfidf_matrix,tfidf_matrix)

In [8]:
indices=pd.Series(df_movies.index,index=df_movies['original_title']).drop_duplicates()
indices

original_title
Avatar                                         0
Pirates of the Caribbean: At World's End       1
Spectre                                        2
The Dark Knight Rises                          3
John Carter                                    4
                                            ... 
El Mariachi                                 4798
Newlyweds                                   4799
Signed, Sealed, Delivered                   4800
Shanghai Calling                            4801
My Date with Drew                           4802
Length: 4803, dtype: int64

In [9]:
indices['Spectre']

2

In [15]:
def get_recommendation(title, cosine_sim=cosine_sim):
    idx=indices[title]
    sim_scores=enumerate(cosine_sim[idx])
    sim_scores=sorted(sim_scores,key=lambda x:x[1],reverse=True)
    sim_scores=sim_scores[1:11]
#     for i in sim_scores:
#         print(i)
    sim_index=[i[0] for i in sim_scores]
    print(sim_index)
    print(df_movies['original_title'].iloc[sim_index])

In [16]:
get_recommendation('Spectre')

[1343, 4071, 11, 29, 3162, 4009, 277, 698, 1717, 4339]
1343    Never Say Never Again
4071    From Russia with Love
11          Quantum of Solace
29                    Skyfall
3162              Thunderball
4009    2016: Obama's America
277             Casino Royale
698       The Prince of Egypt
1717               Safe Haven
4339                   Dr. No
Name: original_title, dtype: object


In [17]:
get_recommendation('The Matrix')

[2900, 1235, 1281, 3504, 4395, 25, 36, 125, 420, 670]
2900            Space Battleship Yamato
1235                               Rent
1281                            Hackers
3504                              11:14
4395                       The Specials
25                              Titanic
36      Transformers: Age of Extinction
125                 The Matrix Reloaded
420         Hellboy II: The Golden Army
670                       Random Hearts
Name: original_title, dtype: object
